<a href="https://colab.research.google.com/github/yogeshrana7/BDDA2/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, plot_roc_curve, classification_report
from  sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_transaction = pd.read_csv("/content/drive/MyDrive/creditcard.csv")
file_transaction.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
file_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
file_transaction.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [ ]:
file_transaction['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [ ]:
file_legit_transaction = file_transaction[file_transaction.Class == 0] 
file_fraud_transaction = file_transaction[file_transaction.Class == 1]
#print(file_fraud_transaction.to_string()) # print the new sub table

In [ ]:
print(file_legit_transaction.shape)
print(file_fraud_transaction.shape)

(284315, 31)
(492, 31)


In [ ]:
# Statistical Measure of the data
file_legit_transaction.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [ ]:
file_fraud_transaction.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [ ]:
file_transaction.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [ ]:
file_legit_transaction_sample = file_legit_transaction.sample(n=492)

In [ ]:
new_file_transaction= pd.concat([file_legit_transaction_sample, file_fraud_transaction], axis = 0)

In [ ]:
new_file_transaction.shape

(984, 31)

In [ ]:
new_file_transaction.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
10573,17526.0,-6.670606,5.163815,-9.484282,4.849274,2.782462,1.029472,-2.424039,4.463098,-2.253184,-2.211725,1.636615,-3.306041,1.721414,-3.109929,0.125672,3.448418,8.538195,4.199591,-0.820060,-0.575025,-0.160400,-0.962718,-1.450336,0.409975,0.501064,0.587661,-0.357686,-0.524894,1.00,0
128046,78580.0,-0.479185,-0.518687,0.458337,-3.631914,1.756679,3.320033,-0.478855,0.587832,-2.445937,1.061418,-0.363671,-1.482894,0.288210,-0.553968,0.254566,-0.044709,-0.292530,0.470494,-0.288407,-0.133713,-0.253976,-0.458342,-0.325432,0.970334,0.517264,-0.305732,-0.253089,-0.150436,15.00,0
225673,144345.0,1.895713,-1.547156,-2.600566,-3.104195,0.453455,0.607453,-0.193953,0.066715,0.515055,-0.403885,1.022519,1.292792,0.326260,0.725984,1.134756,-3.330520,0.528064,0.587929,-0.221167,-0.385963,-0.077579,0.273031,-0.047638,-0.934870,0.183372,-0.632670,0.051896,-0.055097,140.89,0
46479,42805.0,-3.342513,-2.770958,0.923316,1.949572,0.228935,-0.689461,-0.419406,0.806576,0.109976,-0.620545,-1.011700,-0.476969,-1.324442,0.654048,1.126505,-0.352020,0.499346,0.320973,-0.031939,1.456159,0.676662,0.510900,0.635032,0.039481,-0.010673,-0.173436,0.236858,-0.260285,352.00,0


In [ ]:
new_file_transaction['Class'].value_counts()

1    492
0    492
Name: Class, dtype: int64

In [ ]:
new_file_transaction.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,97904.628049,-0.116542,0.114392,-0.035106,-0.055781,-0.046460,-0.032244,-0.049628,-0.050316,0.061658,-0.082586,-0.137419,0.020123,-0.076291,0.004360,-0.026695,0.024537,0.050714,0.006532,0.031809,0.068958,-0.074119,0.008529,0.029564,-0.029777,-0.051888,-0.028561,-0.014724,-0.004440,87.419085
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [ ]:
# Splitting the data into features & target
X = new_file_transaction.drop(columns = 'Class', axis=1)
Y = new_file_transaction['Class']

In [ ]:
X.shape

(984, 30)

In [ ]:
Y.shape

(984,)

In [ ]:
# Split the data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)


In [ ]:
model_lr = LogisticRegression(max_iter=120,random_state=0, n_jobs=20, solver='liblinear')


In [ ]:
model_lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 20.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=120,
                   multi_class='auto', n_jobs=20, penalty='l2', random_state=0,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [ ]:
trn_lr_pred = model_lr.predict(X_train)
trn_lr_acc = accuracy_score(trn_lr_pred, y_train)
print(round(trn_lr_acc*100, 2))

93.02


In [ ]:
tst_lr_pred = model_lr.predict(X_test)
tst_lr_acc = accuracy_score(tst_lr_pred, y_test)
print(round(tst_lr_acc*100, 2))

93.54


In [ ]:
print(classification_report(y_test,tst_lr_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       164
           1       0.97      0.90      0.93       161

    accuracy                           0.94       325
   macro avg       0.94      0.94      0.94       325
weighted avg       0.94      0.94      0.94       325



#####  Conclusion : Model have achieved an accuracy of approx 94% .